<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex26-Binomial_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Economics of Investments: Theory and Data Analytics**, Bates, Boyer, and Fletcher

# Example Chapter 13: Option Pricing with the Binomial Model
This script estimates the no-arbitrage price of a call or put using the binomial option pricing model.

### Imports and Setup

In [ ]:
import numpy as np

In the block of code below we define the function `binomial_tree`  which prices European calls and puts using the binomial method.  

###Function Description
`binomial_tree(type, S0, K, T, r, sigma, n)-> float`
Prices European calls and puts using the binomial tree method.   

**Inputs**

- `type (string)`: option type, must be either `'call'` or `'put'`.
- `S0 (float)`: the current underlying asset price.
- `K (float)`: the strike price.
- `T (float)`: the time to maturity in years.
- `r (float)`: the annualized discrete-time risk-free rate.
- `sigma (float)`: the annualized volatility of the stock return.
- `n (int)`: the number of time steps in the stock tree.



**Returns**  
The price of the designated option (float).


**Example Usage:**
`call_price=('call', 100, 105, 0.50, 0.01, 0.25, 3)`
This gives the no-arbitrage price of a call option on an underlying asset with a current price of $S_T=100$ and volatility of $\sigma=0.25$, strike price $K=105$, that matures in $T=0.5$ years or six months, where the annulized risk-free rate is $1\%$ using 3 steps.


In [ ]:
import numpy as np

def binomial_tree(option_type, S0, K, T, r, sigma, n):
    # length of one time step
    dt = T / n

    # up/down factors (CRR)
    u = np.exp(sigma * np.sqrt(dt))
    d = 1.0 / u

    # one-step discount factor (discrete compounding)
    discfac = (1.0 + r) ** (dt)

    # ------------------------------------------------------------
    # Stock prices at maturity (time n), ORDERED:
    # index 0 = all up moves, index n = all down moves
    #
    # k = number of down moves
    # S_{n,k} = S0 * u^(n-k) * d^k
    # ------------------------------------------------------------
    stock_prices = np.array([S0 * (u ** (n - k)) * (d ** k) for k in range(n + 1)])

    # Option payoffs at maturity in the SAME ORDER
    if option_type == "call":
        option_values = np.maximum(stock_prices - K, 0.0)
    elif option_type == "put":
        option_values = np.maximum(K - stock_prices, 0.0)
    else:
        raise ValueError('Invalid option type. Use "call" or "put".')

    # ------------------------------------------------------------
    # Step backwards through time: i = n-1, ..., 0
    #
    # At time i, there are i+1 nodes (k = 0...i).
    # The relationship between prices across one step is:
    #   S_{i+1,k} = u * S_{i,k}
    # so
    #   S_{i,k} = S_{i+1,k} / u
    # ------------------------------------------------------------
    for i in range(n - 1, -1, -1):
        # Stock prices at time i (keep first i+1 nodes, then go back one step)
        stock_prices = stock_prices[:i + 1] / u

        # For node k at time i:
        #   up child is (i+1, k)   -> option_values[k]
        #   down child is (i+1, k+1)-> option_values[k+1]
        option_up = option_values[:i + 1]
        option_down = option_values[1:i + 2]

        # Delta = (V_up - V_down) / (S_i,k*(u - d))
        delta = (option_up - option_down) / (stock_prices * (u - d))

        # Bond position (chosen to match the UP child, then discounted back)
        B = (option_up - delta * stock_prices * u) / discfac

        # Option value at time i
        option_values = delta * stock_prices + B

    return float(option_values[0])


### Price a Call Option

In [ ]:
# Parameters
S0 = 20  # Initial stock price
K = 21  # Strike price
T = 1/6  # Time to maturity in years
r = 0.01  # Annual risk-free rate
sigma = 0.30  # Annual volatility
n = 2  # Number of time steps

# Calculate option price using the replicating portfolio
call_price = binomial_tree('call',S0, K, T, r, sigma, n)
print(f"Call Option Price: {call_price:.4f}")


### Price a Put Option

In [ ]:
# Parameters
S0 = 20  # Initial stock price
K = 21  # Strike price
T = 1/6  # Time to maturity in years
r = 0.01  # Annual risk-free rate
sigma = 0.30  # Annual volatility
n = 2  # Number of time steps

put_price = binomial_tree('put',S0, K, T, r, sigma, n)
print(f"Put Option Price: {put_price:.4f}")